In [1]:
import numpy as np
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init
import torchvision.datasets as dset
from torchvision import transforms, utils
from torch.utils.data import Dataset, DataLoader
from torch.autograd import Variable
import matplotlib.pyplot as plt
import pickle
from __future__ import print_function, division
from torch.optim import lr_scheduler
from skimage import io, transform

batch_size =16 
#PATH  = "/content/model"
  
print(os.getcwd())


/content


학습데이터 경로

In [2]:
from google.colab import drive

drive.mount('/content/gdrive')

KeyboardInterrupt: ignored

In [72]:
with open('/content/gdrive/My Drive/foo.txt', 'w') as f:
  f.write('Hello Google Drive!')
!cat /content/gdrive/My\ Drive/foo.txt

Hello Google Drive!

In [73]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt

dataset = pd.read_csv('/content/gdrive/My Drive/winequalityN_out.csv')
X_featureSet = dataset.iloc[: , :-1].values
y_qulitySet = dataset.iloc[: , -1:].values
X_featureSet = np.asarray(X_featureSet)
X_featureSet = X_featureSet.astype('float').reshape(-1, 1)

print(dataset.iloc[: , :-1])


print('Image name: {}'.format(y_qulitySet))
print('Landmarks shape: {}'.format(X_featureSet.shape))
print('First 4 Landmarks: {}'.format(X_featureSet[:4]))


      type  fixed acidity  volatile acidity  ...    pH  sulphates  alcohol
0        0            8.5              0.26  ...  3.02       0.50      9.8
1        0            5.8              0.24  ...  3.53       0.43     11.7
2        0            9.1              0.59  ...  3.23       0.38      8.5
3        0            7.1              0.32  ...  3.24       0.40     11.5
4        0            6.9              0.39  ...  3.31       0.37     12.6
...    ...            ...               ...  ...   ...        ...      ...
6474     0            9.1              0.27  ...  3.20       0.46     10.4
6475     0            6.6              0.36  ...  3.41       0.61     12.4
6476     0            7.4              0.24  ...  3.28       0.48     12.5
6477     0            6.9              0.36  ...  3.28       0.36     12.7
6478     0            7.1              0.26  ...  3.37       0.42     12.9

[6479 rows x 12 columns]
Image name: [[0]
 [0]
 [0]
 ...
 [6]
 [6]
 [6]]
Landmarks shape: (77748, 1

In [0]:
class wineDataset(Dataset):
    """Face Landmarks dataset."""

    def __init__(self, csv_file, root_dir, transform=None):
        """
        Args:
            csv_file (string): csv 파일의 경로
            root_dir (string): 모든 이미지가 존재하는 디렉토리 경로
            transform (callable, optional): 샘플에 적용될 Optional transform
        """
        self.wine_frame = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.wine_frame)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

      
        wineFeatures = self.wine_frame.iloc[idx, :-1]
        out = np.zeros((16, 1))
        for index, point in enumerate(wineFeatures):
          out[index+1] = point
        dataSet = np.array([np.dot(out,out.T)])
        wineFeatureSet = dataSet.astype('double')
        
        wineQulitySet = self.wine_frame.iloc[idx, -1:]
        sample = {'wineFeatureSet': wineFeatureSet, 'wineQulitySet': wineQulitySet}
        if self.transform:
            sample = self.transform(sample)

        return sample['wineFeatureSet'], int(sample['wineQulitySet'])


class ToTensor(object):
    """numpy array를 tensor(torch)로 변환 시켜줍니다."""

    def __call__(self, sample):
        wineFeatureSet, wineQulitySet = sample['wineFeatureSet'], sample['wineQulitySet']

        # swap color axis because
        # numpy image: H x W x C
        # torch image: C X H X W
        result = torch.from_numpy(wineFeatureSet)
        result = result.float()
        return {'wineFeatureSet': result,
                'wineQulitySet': wineQulitySet}

In [77]:
wine_dataset = wineDataset(csv_file='/content/gdrive/My Drive/wineQualityTrainNormalization.csv',
                                    root_dir='/content/gdrive/My Drive//',
                                    transform=transforms.Compose([ ToTensor(),
                                                                 # transforms.Normalize(mean=(0.5), std=(0.5)),
                                    
                                    ])
                                    )

wine_testSet =wineDataset(csv_file='/content/gdrive/My Drive/wineQualityTestNormalization.csv',
                                    root_dir='/content/gdrive/My Drive//',
                                    transform=transforms.Compose([ ToTensor()])
                                    )
print (wine_dataset)

image, label = wine_dataset.__getitem__(5)

print(image, label)

image, label = cifar_train.__getitem__(5)

print(image,label)


tensor([[[ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
           0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
           0.0000,  0.0000],
         [ 0.0000,  1.0000,  0.0413, -0.4304, -0.7229, -0.9601, -0.7575,
          -0.9375, -0.9447, -0.5901, -0.2248, -0.5169, -0.6232,  0.0000,
           0.0000,  0.0000],
         [ 0.0000,  0.0413,  0.0017, -0.0178, -0.0299, -0.0397, -0.0313,
          -0.0387, -0.0390, -0.0244, -0.0093, -0.0214, -0.0258,  0.0000,
           0.0000,  0.0000],
         [ 0.0000, -0.4304, -0.0178,  0.1852,  0.3111,  0.4132,  0.3260,
           0.4035,  0.4066,  0.2540,  0.0968,  0.2224,  0.2682,  0.0000,
           0.0000,  0.0000],
         [ 0.0000, -0.7229, -0.0299,  0.3111,  0.5226,  0.6941,  0.5476,
           0.6777,  0.6829,  0.4266,  0.1625,  0.3736,  0.4505,  0.0000,
           0.0000,  0.0000],
         [ 0.0000, -0.9601, -0.0397,  0.4132,  0.6941,  0.9218,  0.7273,
           0.9001,  0.9070,  0.5666,  0.2158,  0.496

CNN 모델에 csv 데이터를 넣기 위한 트라이

```
# 코드로 형식 지정됨
```



In [59]:
#x = np.array(X[0])
out = np.zeros((12, 1))
for index, point in enumerate(X[0]):
  out[index] = point
print(out)
dataSet = torch.from_numpy(out)

print(np.dot(out,out.T))
dataSet = torch.from_numpy(np.dot(out,out.T))
print (dataSet)

NameError: ignored

dset 데이터처럼 만들어 보는 과정

In [34]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)
print(len(X_train))
wine_train = ()
for i in range(0,len(X_train)):
  out = np.zeros((12, 1))
  for index, point in enumerate(X[0]):
    out[index] = point
  dataSet = torch.from_numpy(np.dot(out,out.T))



NameError: ignored

*데이터* 로드, 정규화

In [78]:
cifar_train= dset.CIFAR10("CIFAR10/", train=True,
                          transform=transforms.Compose([
                              transforms.ToTensor(),
                              transforms.Normalize(mean=(0.5,0.5,0.5), std=(0.5,0.5,0.5)),
                            ]),
                          target_transform= None, download=True)
print (cifar_train)
print (len(cifar_train[0][0][0][0]))
print ( len(wine_dataset[0][0][0][0]))
print (len(cifar_train[0][0][0]))
print (len( wine_dataset[0][0][0]))
print (len(cifar_train[0][0]))
print ( len(wine_dataset[0][0]))
print (len(cifar_train[0]))
print ( len(wine_dataset[0]))

cifar_test= dset.CIFAR10("CIFAR10/", train=False, 
                            transform=transforms.Compose([
                                  transforms.ToTensor(),
                                  transforms.Normalize(mean=(0.5,0.5,0.5), std=(0.5,0.5,0.5)),
                            ]),target_transform= None, download=True)

Files already downloaded and verified
Dataset CIFAR10
    Number of datapoints: 50000
    Root location: CIFAR10/
    Split: Train
    StandardTransform
Transform: Compose(
               ToTensor()
               Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
           )
32
16
32
16
3
1
2
2
Files already downloaded and verified


 대략적인 데이터 형태

In [0]:
def ComputeAccr(dloader, imodel):
    correct = 0 
    total= 0 
    
    for j, [imgs, labels] in enumerate(dloader):
        img  = Variable(imgs, volatile = True)
        label = Variable(labels)
        
        output = imodel.forward(img)
        _, output_index = torch.max(output, 1)
        #print(label,output_index)
        total += label.size(0)
        correct += (output_index == label).sum().float()
        
    print("Accuracy of Test Data: {}".format(100*correct/total))
      

def TotalComputeAccr(dloader, imodel):
    classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
    class_correct = list(0. for i in range(10))
    class_total = list(0. for i in range(10))
    
    correct = 0 
    total= 0 
    
    for j, [imgs, labels] in enumerate(dloader):
        img  = Variable(imgs, volatile = True)
        label = Variable(labels)
        
        output = imodel.forward(img)
        _, output_index = torch.max(output, 1)
        c = (output_index == labels).squeeze()
        for i in range(4):
            labelF = label[i]
            class_correct[labelF] += c[i].item()
            class_total[labelF] += 1



        total += label.size(0)
        correct += (output_index == label).sum().float()
        
    #print("Accuracy of Test Data: {}".format(100*correct/total))
    for i in range(10):
      print('Accuracy of %5s : %2d %%' % (
          classes[i], 100 * class_correct[i] / class_total[i]))

데이터 로드 함수

In [0]:
train_loader = torch.utils.data.DataLoader(list(wine_dataset)[:], batch_size= batch_size, shuffle=True, num_workers=2, drop_last=True)
test_loader = torch.utils.data.DataLoader(wine_testSet, batch_size= batch_size, shuffle=False, num_workers=2, drop_last=True)

모델 선언

In [0]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.layer = nn.Sequential(
            nn.Conv2d(1,16,3, padding=1),
            nn.ELU(),
            nn.Dropout2d(0.5),
            #nn.BatchNorm2d(16),
            nn.Conv2d(16,32,3,padding=1),
            nn.ELU(),
            nn.Dropout2d(0.5),
            #nn.BatchNorm2d(32),
            nn.MaxPool2d(2,2),
            nn.Conv2d(32,64,3,padding=1),
            nn.ELU(),
            nn.Dropout2d(0.5),
            #nn.BatchNorm2d(64),
            nn.MaxPool2d(2,2)
        )
        self.fc_layer=nn.Sequential(
            nn.Linear(64*4*4, 64),
            nn.ELU(),
            nn.Dropout2d(0.5),
            nn.BatchNorm1d(64),
            nn.Linear(64,8)
        )
        
        
    def forward(self, x):
        out= self.layer(x)
        out= out.view(batch_size,-1)
        out= self.fc_layer(out)
        return out
    
model= CNN()

loss, optimizer

In [0]:
loss_func=nn.CrossEntropyLoss()
#optimizer=torch.optim.SGD(model.parameters(), lr=learning_rate)

# 불러오기

In [0]:
num_epoch_f=150
#num_epoch_m=100
#num_epoch_l=100

learning_rate = 0.0005
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
#scheduler = lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.2)




epoch = 0
#checkpoint = torch.load("/content/gdrive/My Drive/model_entire_batch64_linear.pkl")
#model.load_state_dict(checkpoint['model_state_dict'])
#optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
#epoch = checkpoint['epoch']
#loss = checkpoint['loss']

#model.eval()
#TotalComputeAccr(test_loader, model)

학습

In [88]:

for i in range(epoch+1 , num_epoch_f):
    print ("%d epoch :" %i) 
    model.train()
    for j,[image, label] in enumerate(train_loader):
        x= Variable(image)
        y_= Variable(label)
        optimizer.zero_grad()
        output=model.forward(x)
        loss=loss_func(output, y_)
        loss.backward()
        optimizer.step()
        if j%1000 == 0:
            print(j,loss)
            # Save the model
            torch.save({
                'epoch': i,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'loss': loss,
          
            }, '/content/gdrive/My Drive/wineQulityModel.pkl')

    model.eval()
    ComputeAccr(test_loader, model)

Accuracy of Test Data: 54.16666793823242
87 epoch :
0 tensor(0.9023, grad_fn=<NllLossBackward>)
Accuracy of Test Data: 56.095680236816406
88 epoch :
0 tensor(1.0463, grad_fn=<NllLossBackward>)


KeyboardInterrupt: ignored

In [89]:
for i in range(12,16):
  print(i)

12
13
14
15


In [13]:
out = np.zeros((16, 1))
print(out)
for index in range(0,12):
    out[index] = index+1
dataSet = np.dot(out,out.T)
memo = np.zeros((16, 16))
print(len(memo[0]))
for i in range(0,16):
  for x in range(0,12):
    for y in range(0,12):
      memo[i*12+x][y] = dataSet[x][y]*out[i]

print(memo)
print(memo[15])


[[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]]
16
[[1.000e+00 2.000e+00 3.000e+00 ... 1.400e+01 1.500e+01 1.600e+01]
 [2.000e+00 4.000e+00 6.000e+00 ... 2.800e+01 3.000e+01 3.200e+01]
 [3.000e+00 6.000e+00 9.000e+00 ... 4.200e+01 4.500e+01 4.800e+01]
 ...
 [2.240e+02 4.480e+02 6.720e+02 ... 3.136e+03 3.360e+03 3.584e+03]
 [2.400e+02 4.800e+02 7.200e+02 ... 3.360e+03 3.600e+03 3.840e+03]
 [2.560e+02 5.120e+02 7.680e+02 ... 3.584e+03 3.840e+03 4.096e+03]]
[ 16.  32.  48.  64.  80.  96. 112. 128. 144. 160. 176. 192. 208. 224.
 240. 256.]
